In [1]:
import praw
from psaw import PushshiftAPI
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import pandas as pd
import datetime as dt

## API

In [2]:
from pmaw import PushshiftAPI
api = PushshiftAPI()

## Select subreddit

In [3]:
subreddit="environment"
limit=1000

# adding timeframe 
before = int(dt.datetime(2022,1,1,1,0).timestamp())
after = int(dt.datetime(2022,1,1,0,0).timestamp())

comments = api.search_comments(subreddit=subreddit, limit=limit, before=before, after=after)
print(f'Retrieved {len(comments)} comments from Pushshift')

Retrieved 26 comments from Pushshift


## Start scraping
Scraping the above selected subreddit for every day. Then saving it in a csv for per year.

In [7]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
temp_df = pd.DataFrame() #Temporary empty dataframe

for m in range(1, 13):
    year = 2021
    yearAfter = year
    monthBefore = m
    monthAfter = m
    day = 32
    minutes = 0
    
    # YEARLY SCRAPE
    # If year has comments < 1000 scrape everything at once 
    scrapeYearDateBefore = int(dt.datetime(year,12,31,0,0).timestamp())
    scrapeYearDateAfter = int(dt.datetime(year,1,1,0,0).timestamp())
       
    # Scrape all comments per Year
    commentsYear = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeYearDateBefore, after=scrapeYearDateAfter)
    
    
    if (len(commentsYear) < 1000): 
        print(f'Retrieved {len(commentsYear)} comments in {year} from Pushshift')
        # convert comments into a datafrom
        comments_df = pd.DataFrame(commentsYear)

        #add all datapoints to a temporary dataframe
        temp_df = temp_df.append(comments_df, ignore_index=True)
    

    
    
    # MONTLY SCRAPE
    else:
        print(f'more than {len(commentsYear)} comments in year {year}')
        endMonthDate = 31
        
        if(m == 1) or (m == 3) or (m == 5) or (m == 7) or (m == 8) or (m == 10) or (m == 12):
            endMonthDate = 31
        elif(m == 2):
            endMonthDate = 28
        else: 
            endMonthDate = 30
            
        # If month has comments < 1000 scrape all monthly data at once 
        scrapeMonthDateBefore = int(dt.datetime(year,m,endMonthDate,0,0).timestamp())
        scrapeMonthDateAfter = int(dt.datetime(year,m,1,0,0).timestamp())
        
        # Scrape all comments per day
        commentsMonth = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeMonthDateBefore, after=scrapeMonthDateAfter)
        
        if (len(commentsMonth) < 1000): 
            print(f'Retrieved {len(commentsMonth)} comments in month: {m} - from Pushshift')
            # convert comments into a datafrom
            comments_df = pd.DataFrame(commentsMonth)

            #add all datapoints to a temporary dataframe
            temp_df = temp_df.append(comments_df, ignore_index=True)

    
    
    
        # DAILY SCRAPE
        else:
            print(f'more than {len(commentsMonth)} comments in month {m} - {year}')
            if (m == 2): #februari 
                day = 29
            if (m == 4) or (m == 6) or (m == 9) or (m == 11): # months with 30 days -> its days +1 for the loop
                day = 31

            for d in range(1, day):
                # date variables
                dayBefore = d
                dayAfter = d-1

                # change dayAfter / monthAfter on certain conditions
                if(dayAfter == 0): # 
                    monthAfter = m-1
                    if(m == 2):
                        dayAfter = 31
                    elif(m==3):
                        dayAfter = 28
                    elif (m == 4) or (m == 6) or (m == 9) or (m == 11) or (m == 13):
                        dayAfter = 31
                    else: dayAfter = 30
                else: monthAfter = m

                if(monthAfter == 0):
                    continue

                #set time frame for scraping -> Scraping for every day in the year.
                scrapeDayDateBefore = int(dt.datetime(year,monthBefore,dayBefore,0,0).timestamp())
                scrapeDayDateAfter = int(dt.datetime(yearAfter,monthAfter,dayAfter,0,0).timestamp())


                # Scrape all comments per day
                commentsDay = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeDayDateBefore, after=scrapeDayDateAfter)
                  
                # if statement day comments < 1000
                if (len(commentsDay) < 1000): 
                    print('datebefore', dayBefore, '-', monthBefore, '- ', year, 'date after', dayAfter, '-', monthAfter, '- ', yearAfter)
                    print(f'Retrieved {len(commentsDay)} comments per day from Pushshift')
                    # convert comments into a datafrom
                    comments_df = pd.DataFrame(commentsDay)

                    #add all datapoints to a temporary dataframe
                    temp_df = temp_df.append(comments_df, ignore_index=True)
      
    
    
    
                # HOURLY SCRAPE
                else: 
                    print(f'more than {len(commentsDay)} comments on day {dayAfter}-{monthAfter}-{yearAfter} -- {dayBefore}-{monthBefore}-{year}')

                    for h in range(24):

                        # add an hour to the datetime object
                        hourBefore = h + 1
                        hourAfter = h
                        minuteAfter = 0

                        if (hourBefore == 24):
                            hourBefore = 23
                            minutes = 59
                        else: 
                            hourBefore = h + 1
                            minutes = 0
                            


                        #set time frame for scraping -> Scraping for every hour in the day
                        scrapeHourDateBefore = int(dt.datetime(year,m,d,hourBefore,minutes).timestamp())
                        scrapeHourDateAfter = int(dt.datetime(yearAfter,m,d,hourAfter,minuteAfter).timestamp())      
                        
                        # Scrape all comments per hour
                        commentsHour = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeHourDateBefore, after=scrapeHourDateAfter)
                        
                        # if statement hour comments < 1000
                        if (len(commentsHour) < 1000): 
                            print('datebefore', dt.datetime(year,m,d,hourBefore,minutes), 'date after', dt.datetime(yearAfter,m,d,hourAfter,minuteAfter)) 
                            print(f'Retrieved {len(commentsHour)} comments per hour from Pushshift')
                            # convert comments into a datafrom
                            comments_df = pd.DataFrame(commentsHour)

                            #add all datapoints to a temporary dataframe
                            temp_df = temp_df.append(comments_df, ignore_index=True)

                        # HOURLY SCRAPE
                        else:
                            # convert comments into a datafrom
                            comments_df = pd.DataFrame(commentsHour)

                            #add all datapoints to a temporary dataframe
                            temp_df = temp_df.append(comments_df, ignore_index=True)
                            print('ERROR MORE THAN 1000, however, still going on. INCOMPLETE')
                            print(f'more than {len(commentsHour)} comments on day {d}-{m}-{year} between',  dt.datetime(year,m,d,hourBefore,minutes), '-', dt.datetime(yearAfter,m,d,hourAfter,minuteAfter))
#                             raise ValueError("More than 1000 comments in this HOUR.")

                                
                                
                                
                                
                                

# all datapoints in temporary dataframe to one dataframe
df1 = pd.DataFrame(temp_df)
                                      
# create the filename with the variable name embedded
filename = f'{subreddit}_{year}.csv'

# save the dataframe to the file with the embedded variable name
df1.to_csv(filename, encoding='utf-8', header=True, index=False)


more than 1000 comments in year 2021
more than 1000 comments in month 1 - 2021
datebefore 2 - 1 -  2021 date after 1 - 1 -  2021
Retrieved 462 comments per day from Pushshift
datebefore 3 - 1 -  2021 date after 2 - 1 -  2021
Retrieved 344 comments per day from Pushshift
datebefore 4 - 1 -  2021 date after 3 - 1 -  2021
Retrieved 363 comments per day from Pushshift
datebefore 5 - 1 -  2021 date after 4 - 1 -  2021
Retrieved 270 comments per day from Pushshift
datebefore 6 - 1 -  2021 date after 5 - 1 -  2021
Retrieved 264 comments per day from Pushshift
datebefore 7 - 1 -  2021 date after 6 - 1 -  2021
Retrieved 302 comments per day from Pushshift
datebefore 8 - 1 -  2021 date after 7 - 1 -  2021
Retrieved 134 comments per day from Pushshift
datebefore 9 - 1 -  2021 date after 8 - 1 -  2021
Retrieved 177 comments per day from Pushshift
datebefore 10 - 1 -  2021 date after 9 - 1 -  2021
Retrieved 228 comments per day from Pushshift
datebefore 11 - 1 -  2021 date after 10 - 1 -  2021
Retr

datebefore 25 - 3 -  2021 date after 24 - 3 -  2021
Retrieved 418 comments per day from Pushshift
datebefore 26 - 3 -  2021 date after 25 - 3 -  2021
Retrieved 351 comments per day from Pushshift
datebefore 27 - 3 -  2021 date after 26 - 3 -  2021
Retrieved 313 comments per day from Pushshift
datebefore 28 - 3 -  2021 date after 27 - 3 -  2021
Retrieved 339 comments per day from Pushshift
datebefore 29 - 3 -  2021 date after 28 - 3 -  2021
Retrieved 348 comments per day from Pushshift
datebefore 30 - 3 -  2021 date after 29 - 3 -  2021
Retrieved 406 comments per day from Pushshift
datebefore 31 - 3 -  2021 date after 30 - 3 -  2021
Retrieved 391 comments per day from Pushshift
more than 1000 comments in year 2021
more than 1000 comments in month 4 - 2021
datebefore 1 - 4 -  2021 date after 31 - 3 -  2021
Retrieved 543 comments per day from Pushshift
datebefore 2 - 4 -  2021 date after 1 - 4 -  2021
Retrieved 259 comments per day from Pushshift
datebefore 3 - 4 -  2021 date after 2 - 4 

datebefore 2021-04-27 23:00:00 date after 2021-04-27 22:00:00
Retrieved 38 comments per hour from Pushshift
datebefore 2021-04-27 23:59:00 date after 2021-04-27 23:00:00
Retrieved 33 comments per hour from Pushshift
datebefore 28 - 4 -  2021 date after 27 - 4 -  2021
Retrieved 586 comments per day from Pushshift
datebefore 29 - 4 -  2021 date after 28 - 4 -  2021
Retrieved 479 comments per day from Pushshift
datebefore 30 - 4 -  2021 date after 29 - 4 -  2021
Retrieved 567 comments per day from Pushshift
more than 1000 comments in year 2021
more than 1000 comments in month 5 - 2021
datebefore 1 - 5 -  2021 date after 30 - 4 -  2021
Retrieved 426 comments per day from Pushshift
datebefore 2 - 5 -  2021 date after 1 - 5 -  2021
Retrieved 445 comments per day from Pushshift
datebefore 3 - 5 -  2021 date after 2 - 5 -  2021
Retrieved 818 comments per day from Pushshift
datebefore 4 - 5 -  2021 date after 3 - 5 -  2021
Retrieved 527 comments per day from Pushshift
datebefore 5 - 5 -  2021 d

datebefore 17 - 7 -  2021 date after 16 - 7 -  2021
Retrieved 510 comments per day from Pushshift
datebefore 18 - 7 -  2021 date after 17 - 7 -  2021
Retrieved 604 comments per day from Pushshift
datebefore 19 - 7 -  2021 date after 18 - 7 -  2021
Retrieved 684 comments per day from Pushshift
datebefore 20 - 7 -  2021 date after 19 - 7 -  2021
Retrieved 497 comments per day from Pushshift
datebefore 21 - 7 -  2021 date after 20 - 7 -  2021
Retrieved 474 comments per day from Pushshift
datebefore 22 - 7 -  2021 date after 21 - 7 -  2021
Retrieved 492 comments per day from Pushshift
datebefore 23 - 7 -  2021 date after 22 - 7 -  2021
Retrieved 603 comments per day from Pushshift
datebefore 24 - 7 -  2021 date after 23 - 7 -  2021
Retrieved 529 comments per day from Pushshift
datebefore 25 - 7 -  2021 date after 24 - 7 -  2021
Retrieved 557 comments per day from Pushshift
datebefore 26 - 7 -  2021 date after 25 - 7 -  2021
Retrieved 352 comments per day from Pushshift
more than 1000 comme

Not all PushShift shards are active. Query results may be incomplete.


...

Not all PushShift shards are active. Query results may be incomplete.


datebefore 19 - 9 -  2021 date after 18 - 9 -  2021
Retrieved 0 comments per day from Pushshift
datebefore 20 - 9 -  2021 date after 19 - 9 -  2021
Retrieved 557 comments per day from Pushshift
datebefore 21 - 9 -  2021 date after 20 - 9 -  2021
Retrieved 535 comments per day from Pushshift
datebefore 22 - 9 -  2021 date after 21 - 9 -  2021
Retrieved 662 comments per day from Pushshift
datebefore 23 - 9 -  2021 date after 22 - 9 -  2021
Retrieved 468 comments per day from Pushshift
datebefore 24 - 9 -  2021 date after 23 - 9 -  2021
Retrieved 420 comments per day from Pushshift
datebefore 25 - 9 -  2021 date after 24 - 9 -  2021
Retrieved 639 comments per day from Pushshift
datebefore 26 - 9 -  2021 date after 25 - 9 -  2021
Retrieved 422 comments per day from Pushshift
datebefore 27 - 9 -  2021 date after 26 - 9 -  2021
Retrieved 626 comments per day from Pushshift
datebefore 28 - 9 -  2021 date after 27 - 9 -  2021
Retrieved 565 comments per day from Pushshift
datebefore 29 - 9 -  2

Not all PushShift shards are active. Query results may be incomplete.


datebefore 16 - 11 -  2021 date after 15 - 11 -  2021
Retrieved 0 comments per day from Pushshift
datebefore 17 - 11 -  2021 date after 16 - 11 -  2021
Retrieved 580 comments per day from Pushshift
datebefore 18 - 11 -  2021 date after 17 - 11 -  2021
Retrieved 573 comments per day from Pushshift
more than 1000 comments on day 18-11-2021 -- 19-11-2021
datebefore 2021-11-19 01:00:00 date after 2021-11-19 00:00:00
Retrieved 50 comments per hour from Pushshift
datebefore 2021-11-19 02:00:00 date after 2021-11-19 01:00:00
Retrieved 36 comments per hour from Pushshift
datebefore 2021-11-19 03:00:00 date after 2021-11-19 02:00:00
Retrieved 18 comments per hour from Pushshift
datebefore 2021-11-19 04:00:00 date after 2021-11-19 03:00:00
Retrieved 26 comments per hour from Pushshift
datebefore 2021-11-19 05:00:00 date after 2021-11-19 04:00:00
Retrieved 13 comments per hour from Pushshift
datebefore 2021-11-19 06:00:00 date after 2021-11-19 05:00:00
Retrieved 15 comments per hour from Pushshif

datebefore 2021-12-29 13:00:00 date after 2021-12-29 12:00:00
Retrieved 19 comments per hour from Pushshift
datebefore 2021-12-29 14:00:00 date after 2021-12-29 13:00:00
Retrieved 24 comments per hour from Pushshift
datebefore 2021-12-29 15:00:00 date after 2021-12-29 14:00:00
Retrieved 29 comments per hour from Pushshift
datebefore 2021-12-29 16:00:00 date after 2021-12-29 15:00:00
Retrieved 22 comments per hour from Pushshift
datebefore 2021-12-29 17:00:00 date after 2021-12-29 16:00:00
Retrieved 25 comments per hour from Pushshift
datebefore 2021-12-29 18:00:00 date after 2021-12-29 17:00:00
Retrieved 37 comments per hour from Pushshift
datebefore 2021-12-29 19:00:00 date after 2021-12-29 18:00:00
Retrieved 40 comments per hour from Pushshift
datebefore 2021-12-29 20:00:00 date after 2021-12-29 19:00:00
Retrieved 56 comments per hour from Pushshift
datebefore 2021-12-29 21:00:00 date after 2021-12-29 20:00:00
Retrieved 59 comments per hour from Pushshift
datebefore 2021-12-29 22:00:

print(len(df1))

In [8]:
print(len(df1))

170093
